# 실습: 웹크롤링

### 웹 크롤링에 필요한 라이브러리 설치

In [ ]:
# 정적 크롤링을 위한 requests 설치
!pip install requests   

In [ ]:
# HTML과 XML 문서를 파싱하기 위한 파이썬 패키지
!pip install beautifulsoup4 

In [ ]:
# 동적 크롤링을 위한 셀레니움 설치
!pip install selenium      

### requests 사용

In [1]:
import requests

requests.get("https://google.com")

<Response [200]>

In [2]:
import requests

response = requests.get("https://google.com")

# 응답 상태
print('#응답 상태: ', response.status_code)

# 응답 바이너리 원문
print('#응답 바이너리 원문: ', response.content)

# 응답 UTF-8로 인코딩된 문자열
print('#응답 UTF-8로 인코딩된 문자열: ', response.text)

# 응답 헤더
print('#응답 헤더: ', response.headers)

# 응답 헤더: 콘텐트 유형
print('#응답 헤더유형: ', response.headers['Content-Type'])

#응답 상태:  200
#응답 바이너리 원문:  b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="zzTu7MY1oz2W8qVt9iispA">(function(){window.google={kEI:\'Q1cgY_mmB6SSr7wPtfWT2Ac\',kEXPI:\'0,1302530,56879,1709,4349,207,4804,2316,383,246,5,5367,1123753,1197780,614,80,380017,16114,28684,22431,1361,283,12033,2819,1930,12834,4998,13226,3849,10623,22740,5081,1593,1279,2742,149,1103,840,6297,108,3406,606,2023,1777,520,14670,3227,2847,5,24989,8781,1851,2614,13142,3,576,6460,148,13975,4,1528,2304,7039,6344,18729,2658,7357,13658,4437,16786,5800,2557,4094,17,4035,3,3541,1,26959,15201,2,14016,6248,4538,3330,11623,5679,1021,2377,28744,4568,6259,23418,1252,5835,14967,4333,19,4997,2468,445,2,2,1,6960,10346,9326,8049,106,6582,799,14680,1289,873,6578,8259,4797,7,1782,140,9779,21779,2644,689

----------------

## 정적 웹크롤링

### [실습문제] 네이버 뉴스 검색 (Tag Parsing)

#### 1.네이버 뉴스검색 결과를 웹크롤링으로 가져오기
- DataFrame : 기사제목, URL 

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


keyword = input('검색 키워드를 입력하세요 : ')
query = keyword.replace(' ', '+')

print('-'*30)
print("뉴스 데이터 크롤링하기...")
#-------------------------------
news_count, total = 100, 100
news_url = 'https://search.naver.com/search.naver?where=news&query={}&start='
titles, urls = [], []
while news_count: 
    req = requests.get(news_url.format(query) + str(len(titles)))
    soup = BeautifulSoup(req.text, 'html.parser')
    page_tags   = soup.select('ul.list_news > li > div > div > a')
    page_titles = list(map(lambda tag: tag.get('title'), page_tags))
    page_urls   = list(map(lambda tag: tag.get('href'), page_tags))
#-------------------------------
    if len(page_titles) < news_count:
        titles += page_titles
        urls += page_urls
        news_count -= len(page_titles) 
    else: 
        titles += page_titles[:news_count]
        urls += page_urls[:news_count]
        news_count = 0 

    print(f"{total - news_count}% ", end="")
print()
print('-'*30) 
print(f'"{keyword}" 검색어로 검색된 뉴스 [{total}]건 가져오기 완료!')  
print('-'*30)


# DataFrame + Series => DataFrame으로 나타내기
df = pd.DataFrame(titles, columns=['title'])
sr = pd.Series(urls)
df['url'] = sr

df

검색 키워드를 입력하세요 : 고환율
------------------------------
뉴스 데이터 크롤링하기...
10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
------------------------------
"고환율" 검색어로 검색된 뉴스 [100]건 가져오기 완료!
------------------------------


,title,url
0,"""한국의 과도한 긴축이 고환율 부른다""",https://view.asiae.co.kr/article/2022091307383292976
1,저성장·고금리·고환율…하반기 경제 ‘복합 위기’ 비상등,https://www.khan.co.kr/economy/economy-general/article/202209122107005
2,고환율·무역수지 악화...하반기 역성장 쇼크 우려,https://www.dailian.co.kr/news/view/1151118/?sc=Naver
3,"'李 사법리스크'속 野 민생대책위 출범…""경제위기 대안 제시""",https://www.yna.co.kr/view/AKR20220913120200001?input=1195m
4,고환율 장기화에…업종별 희비 엇갈리는 산업계,https://news.mtn.co.kr/news-detail/2022091314334950721
...,...,...
95,"[韓경제 암초 `고환율`] `2009년 금융위기` 원화가치인데… ""괜찮다""는 정부 믿어도 되나",http://www.dt.co.kr/contents.html?article_no=2022090502100363056001&ref=naver
96,하늘길 장벽 낮아진 여행객들...고환율·고유가 여전히 걸림돌,https://www.ytn.co.kr/_ln/0102_202209012243230416
97,고환율·고물가에 얼어붙은 추석 체감 경기,https://www.cjb.co.kr/home/sub.php?menukey=61&mod=view&P_NO=220902522&PRO_CODE=99
98,소상공인 고금리‧고환율‧고물가 3高 대응…58조원 규모 신규·대환대출,https://www.news1.kr/articles/4782768


In [4]:
# 파일로 저장하기 (1000건 추출하여 저장하여라!!!)
file = f'data/naver_news_{keyword}.csv'
# file = 'data/naver_news.csv'
news_df = df['title']
news_df.to_csv(file, index=False, encoding="utf-8-sig")

-----------------------

## 동적 페이지 크롤링 - Selenium


### # selenium webdriver 다운로드(for chrome) 

- https://chromedriver.chromium.org/
- 최신 chrome webdriver 다운로드
- 해당 위치에 WebDriver폴더 만들고 exe파일 옮겨놓는다.
-  (C:/python/projectmanager/WebDriver/chromedriver.exe)
- 자신의 크롬 웹 브라우저의 버전을 확인하고 버전에 맞는 것을 다운로드해야한다.

### # webdriver 동작 테스트하기
- 자신의 크롬 웹 브라우저의 버전을 확인하고 버전에 맞는 것을 다운로드해야한다. 그렇지 않으면 오류가 발생한다.

In [7]:
from selenium import webdriver

driver = 'C:/python/projectmanager/WebDriver/chromedriver.exe'
wd = webdriver.Chrome(driver)

wd.get('https://www.naver.com/')

wd.close()  # 브라우저가 실행되었다가 자동으로 닫힌다.

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16408\3097423985.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(driver)


### [실습]  커피빈매장 정보 크롤링하여 파일로 저장하기

- 아래 사이트를 이용해 호출해야할 자바스크립트 함수를 확인하다.
- https://www.coffeebeankorea.com
- https://www.coffeebeankorea.com/store/store.asp
- (매장 번호로) 자세히보기: javascript:storePop2('374'); 

In [8]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import datetime

from selenium import webdriver
import time

MAX = 50   # 전체 매장 중 일부만 실행해 본다.(10~100개)
#[CODE 1]
def CoffeeBean_store(result):
    CoffeeBean_URL = "https://www.coffeebeankorea.com/store/store.asp"
    wd = webdriver.Chrome('./WebDriver/chromedriver.exe')
    
    total, cnt = MAX, 0         
    for i in range(1, total+1):  #매장 수 만큼(370) 반복        
        wd.get(CoffeeBean_URL)
        time.sleep(1)  #웹페이지 연결할 동안 1초 대기
        try:
            wd.execute_script("storePop2(%d)" %i)
            time.sleep(1) #스크립트 실행 할 동안 1초 대기
            
            html = wd.page_source
            soupCB = BeautifulSoup(html, 'html.parser')
            store_name_h2 = soupCB.select("div.store_txt > h2")
            store_name = store_name_h2[0].string  #매장 이름
            
            store_info = soupCB.select("div.store_txt > table.store_table > tbody > tr > td")
            store_address_list = list(store_info[2])
            store_address = store_address_list[0]  #매장 주소
            
            store_phone = store_info[3].string     #매장 전화번호
            result.append([store_name]+[store_address]+[store_phone])  
            cnt += 1
            # 매장정보 가져온 데이터 출력하기
            print("[%3d] %3d - %s" % (cnt, i, store_name))
            
        except:
            continue 
    return

#[CODE 0]
def main():
    result = []
    print('CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>')
    CoffeeBean_store(result)  #[CODE 1]
    
    CB_tbl = pd.DataFrame(result, columns=('store', 'address','phone'))
    CB_tbl.to_csv('c:/python/highschool/data/CoffeeBean2.csv', encoding='cp949', mode='w', index=True)

if __name__ == '__main__':
     main()

CoffeeBean store crawling >>>>>>>>>>>>>>>>>>>>>>>>>>


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_16408\4141551279.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('./WebDriver/chromedriver.exe')


[  1]   1 - 학동역 DT점
[  2]   3 - 차병원점
[  3]   6 - 강남대로점
[  4]  11 - 강남에스점
[  5]  12 - 청담에스점
[  6]  13 - 신사점
[  7]  14 - 압구정역점
[  8]  15 - 역삼점
[  9]  16 - 양재스포타임점
[ 10]  17 - 청담성당점
[ 11]  18 - 영동점
[ 12]  19 - 도곡점
[ 13]  20 - 영동고앞점
[ 14]  25 - 압구정시티점
[ 15]  26 - 압구정로데오점
[ 16]  27 - 서초우성점
[ 17]  28 - 논현팍스타워점
[ 18]  29 - 삼성오크우드점
[ 19]  30 - 트레이드타워점
[ 20]  31 - 삼성봉은사거리점
[ 21]  34 - 반포엘루체점
[ 22]  35 - 잠실신천점
[ 23]  36 - 서초지파이브점
[ 24]  38 - 논현동수면센터점
[ 25]  41 - 삼성전자강남사옥점
[ 26]  44 - 삼성생명강남사옥점
[ 27]  45 - 방배카페골목점
[ 28]  46 - 역삼포스틸타워뒷점
[ 29]  47 - 테헤란로하이닉스뒷점
[ 30]  49 - 삼성봉은사로점
[ 31]  50 - 방이점


-------------------------------

## OpenAPI 이용한 검색

### [실습] 네이버 책검색 오픈API 코드 --> DataFrame --> csv파일로 저장

In [11]:
import os
import sys
import urllib.request
client_id = "WlIpA7IkqcaBRP6XKyUk"
client_secret = "pIew6Mp_Fr"

title = input("검색 도서명: ")
encText = urllib.parse.quote(title)

# 책 검색 URL(json)
url = "https://openapi.naver.com/v1/search/book.json?query=" + encText 
# 블로그 검색 URL(json)
# url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
# 블로그 검색 URL(xml)
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

검색 도서명: 파이썬
{
	"lastBuildDate":"Tue, 13 Sep 2022 19:21:00 +0900",
	"total":779,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"전문가를 위한 파이썬 프로그래밍 (애플리케이션 구축,유지보수,패키징,배포 등 모던 파이썬 개발 마스터하기)",
			"link":"https:\/\/search.shopping.naver.com\/book\/catalog\/33762315620",
			"image":"https:\/\/shopping-phinf.pstatic.net\/main_3376231\/33762315620.20220817093949.jpg",
			"author":"미하우 야보르스키^타레크 지아데",
			"discount":"34200",
			"publisher":"제이펍",
			"pubdate":"20220812",
			"isbn":"9791192469201",
			"description":"베테랑에게 배우는 모던 파이썬 개발의 베스트 프랙티스와 인사이트\n\n프로덕션 레벨에서 파이썬을 파이썬답게 쓰기 위해 알아야 할 모든 것을 망라한 책. 최신 피처를 활용해 파이썬 애플리케이션을 구축, 유지보수, 패키징, 배포하는 방법을 깊숙이 살펴본다. 베테랑 개발자가 엄선한 베스트 프랙티스, 유용한 도구, 표준을 익힘으로써 파이썬 전문가에 한 걸음 더 가까워질 수 있다."
		},
		{
			"title":"혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)",
			"link":"https:\/\/search.shopping.naver.com\/book\/catalog\/32507605957",
			"image":"https:\/\/shopping-phinf.pstatic.net\/main_3250760\/32507605957.20220603093703.jpg",
			"author":"윤인성",
			"discount":"195

### [미션] 사용자가 입력한 검색어 + 사용자가 입력한 건수 만큼 가져와서 DataFrame과 파일로 저장하기

In [12]:
import urllib.request
import json

#------------------------------------
# 네이버 애플리케이션 등록 정보
#------------------------------------
Client_ID = "WlIpA7IkqcaBRP6XKyUk"  # 자신의 client id 
Client_Secret = "pIew6Mp_Fr"    # 자신의 client secret


#------------------------------------
#  검색어 입력받아 네이버검색 url 설정
#------------------------------------
# 검색어 입력
title = input("검색 도서명: ")

# 검색어 URL에 추가하기
query = urllib.parse.quote(title)
query = "https://openapi.naver.com/v1/search/book.json?query=" + query               
# 가져올 데이터 개수 설정하기
total = 10
url_query= query + f"&display={total}"

print('요청 URL: ', url_query)

    
#Open API 검색 요청 개체 설정
request = urllib.request.Request(url_query)
request.add_header("X-Naver-Client-Id", Client_ID)
request.add_header("X-Naver-Client-Secret", Client_Secret)


#------------------------------------
# 검색 요청 및 처리
#------------------------------------
response = urllib.request.urlopen(request)
rescode = response.getcode()

if(rescode == 200):
    result = response.read().decode('utf-8')
    #print(result) # result에 어떤 값이 들어 있는지 확인해 보기
else:
    print("검색에 실패하였습니다.")
    exit()

#------------------------------------
# 검색 결과 parsing 해서 보기좋게 출력하기
#------------------------------------
j_result = json.loads(result)   #검색 결과를 json타입으로 보여주기
if j_result == None:
    print("json.loads 실패하였습니다.")
    exit()
#print(j_result) # j_result에 어떤 값이 들어 있는지 확인해 보기
 
# j_result의 결과 중 도서 정보가 담긴 items 항목의 내용을 parsing한다.
books = dict()  # 데이터가 저장될 변수
for item in j_result['items']:
    # 검색된 책 딕셔너리로 만들기 
    # key 정보
    key = item['title']    
    # value
    tmp = list()
    tmp.append(item['author'])
    tmp.append(item['link'])
    # 딕셔너리에 책 제목과 값을 추가
    books[ key ] = tmp
    print('키: ', key)
    print('--:', tmp)
    print('-'*30)
    
#     print("\n제목: " + item['title'])
#     print("저자: " + item['author'])
#     print("설명: " + item['description'])
#     print("url: " + item['link'])
#     print("\n")
    
#

검색 도서명: 파이썬
요청 URL:  https://openapi.naver.com/v1/search/book.json?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC&display=10
키:  전문가를 위한 파이썬 프로그래밍 (애플리케이션 구축,유지보수,패키징,배포 등 모던 파이썬 개발 마스터하기)
--: ['미하우 야보르스키^타레크 지아데', 'https://search.shopping.naver.com/book/catalog/33762315620']
------------------------------
키:  혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서)
--: ['윤인성', 'https://search.shopping.naver.com/book/catalog/32507605957']
------------------------------
키:  클린 코드, 이제는 파이썬이다 (한 권으로 읽는 파이썬 개발자 성장 프로젝트)
--: ['알 스웨이가트', 'https://search.shopping.naver.com/book/catalog/33881813178']
------------------------------
키:  파이썬과 40개의 작품들 (자동화, 크롤링, 이미지처리, 데이터분석, 웹페이지, GUI 프로그램, 게임)
--: ['장문철', 'https://search.shopping.naver.com/book/catalog/32472041705']
------------------------------
키:  CODING BASICS PYTHON (파이썬)
--: ['김상민^장성식^김일태', 'https://search.shopping.naver.com/book/catalog/32440944646']
------------------------------
키:  파이썬 머신러닝 완벽 가이드 (다양한 캐글 예제와 함께 기초 알고리즘부터 최신 기법까지 배우는)
--: ['권철민', 'https://search.shop

In [13]:
#------------------------------------
# DataFrame으로 만들기
#------------------------------------   
import pandas as pd

df = pd.DataFrame(books)
df = df.T              # 행과 열이 바뀐다.
df = df.reset_index()  # 인덱스 초기화
df.columns = ['책제목', '저자','URL']  # 컬럼명 지정
df

#------------------------------------
# DataFrame-->파일로 저장하기
#------------------------------------   
file = 'data/naver_books.csv'
df.to_csv(file)

df = pd.read_csv(file)
df.drop(['Unnamed: 0'], axis=1, inplace=True )
df

,책제목,저자,URL
0,"전문가를 위한 파이썬 프로그래밍 (애플리케이션 구축,유지보수,패키징,배포 등 모던 파이썬 개발 마스터하기)",미하우 야보르스키^타레크 지아데,https://search.shopping.naver.com/book/catalog/33762315620
1,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),윤인성,https://search.shopping.naver.com/book/catalog/32507605957
2,"클린 코드, 이제는 파이썬이다 (한 권으로 읽는 파이썬 개발자 성장 프로젝트)",알 스웨이가트,https://search.shopping.naver.com/book/catalog/33881813178
3,"파이썬과 40개의 작품들 (자동화, 크롤링, 이미지처리, 데이터분석, 웹페이지, GUI 프로그램, 게임)",장문철,https://search.shopping.naver.com/book/catalog/32472041705
4,CODING BASICS PYTHON (파이썬),김상민^장성식^김일태,https://search.shopping.naver.com/book/catalog/32440944646
5,파이썬 머신러닝 완벽 가이드 (다양한 캐글 예제와 함께 기초 알고리즘부터 최신 기법까지 배우는),권철민,https://search.shopping.naver.com/book/catalog/32485894885
6,파이썬,홍의경,https://search.shopping.naver.com/book/catalog/32460987987
7,Do it! 점프 투 파이썬 (이미 200만명이 이 책으로 프로그래밍을 시작했다!),박응용,https://search.shopping.naver.com/book/catalog/32456895000
8,파이썬 (제2판),염기원^오지영,https://search.shopping.naver.com/book/catalog/32436240934
9,파이썬 알고리즘 인터뷰 (95가지 알고리즘 문제 풀이로 완성하는 코딩 테스트),박상길,https://search.shopping.naver.com/book/catalog/32456486633


------------------------

### [실습] 네이버 뉴스 검색 API 코드 --> 파일로 저장
- 파일명: naver_news_키워드.json, naver_news_키워드.csv

In [14]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

client_id = '6jnPJVA9p5vOCZgi9PGg'  # 자신의 API ID
client_secret = '_ShPf0e05n'        # 자신의 API PASSWORD

news = []   #csv파일을 위한 변수

#[CODE 1]
#  URL 실행(요청)하기
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
# Naver OpenAPI 검색URL 만들어 요청하기
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
#가져온 Json 데이터 분리하기
def getPostData(post, jsonResult, cnt): 
    title = post['title']
    description = post['description']
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description}) 
    
    news.append({'cnt':cnt, 'title':title, 'description': description})
    
    return    


#[CODE 0]
# 메인 함수
def main():
    node = 'news'   # 크롤링 할 대상 : news / blog
    total = 100
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, total)  #[CODE 2]
    total = jsonResponse['total']
 
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  #[CODE 3]       
        
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, total)  #[CODE 2]
       
    print('전체 검색 : %d 건' %total)
    
    with open('naver_%s_%s.json' % (node, srcText), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('naver_%s_%s.json SAVED' % (node, srcText))
    

    # csv 파일로 저장하기
    df = pd.DataFrame(news)
    file = 'data/naver_%s_%s.csv' % (node, srcText)
    df.to_csv(file, index=False, encoding="utf-8-sig")
    
#-----------------------    
# 메인함수 호출
#-----------------------  
main()


검색어를 입력하세요: 고환율
[2022-09-13 19:24:50.888684] Url Request Success
HTTP Error 400: Bad Request
[2022-09-13 19:24:51.027498] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EA%B3%A0%ED%99%98%EC%9C%A8&start=101&display=34517
전체 검색 : 34517 건
가져온 데이터 : 100 건
naver_news_고환율.json SAVED
